# Text Abstractions: Articles Skimming

In [1]:
# Importing TF and checking the version
import tensorflow as tf

print(tf.__version__)

2.10.1


Re-using PubMed dataset, published by Franck Dernoncourt. Dataset contains 20K records, with numbers replaced by "@" sign.

In [2]:
# Checking all the file names in the target directory
import os

data_dir = "PubMed_20K_RCT_NumReplaced/"
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

['PubMed_20K_RCT_NumReplaced/dev.txt',
 'PubMed_20K_RCT_NumReplaced/test.txt',
 'PubMed_20K_RCT_NumReplaced/train.txt']